In [1]:
import pandas as pd
import numpy as np

### Funkcije koje menjaju imena celija, izbacuju nevalidne gene, izbacuju celije koje nisu korisne 

In [2]:
def rename_cells(df, cnum, file_name, i):
    file_number = file_name[0 : file_name.index('_')+1]
    columns=['Index']
    for j in range(1, cnum):
        column_name =file_number + str(j)
        columns.append(column_name)
    df.columns = columns
    
    return df

In [3]:
def remove_invalid_genes(df, valid_genes_list, gnum):
    df_genes = list(df["Index"])
    g_index = []
    for i in range(0, gnum):
        if df_genes[i] not in valid_genes_list:
            g_index.append(i)
    df = df.drop(g_index)
    df = df.reset_index(drop=True)
    
    return df

In [4]:
def get_num_and_sum_of_positive_cells(df):
    columns = list(df.columns)[1:]
    num_of_positive_list = [] 
    sum_of_positive_list = []
    for column in columns:
        values = list(df[column])
        
        num_of_positive = len(values) - values.count(0)
        num_of_positive_list.append(num_of_positive)
        
        sum_of_positive = sum(values)
        sum_of_positive_list.append(sum_of_positive)
        
    return (num_of_positive_list, sum_of_positive_list)

In [5]:
def remove_cells(df, cnum, num_of_positive, sum_of_positive): 
    (num_of_positive_list, sum_of_positive_list) = get_num_and_sum_of_positive_cells(df)
    print(len(num_of_positive_list))
    columns = list(df.columns)
    cell_names=[]
    for i in range(len(num_of_positive_list)):
        if num_of_positive_list[i] < num_of_positive or sum_of_positive_list[i] < sum_of_positive:
            cell_names.append(columns[i+1])
    print(len(cell_names))
    df = df.drop(cell_names, axis=1)
    
    return df

### Izvlacenje broja pozitivnih celija za svaki gen u svakoj datoteci u fajl Num_of_positive_per_gene.csv

In [6]:
def genes_info(valid_genes_list, files):
    
    #valid_genes_list.sort()
    columns=['Index', 'GSM3892570', 'GSM3892571', 'GSM3892572', 'GSM3892573', 'GSM3892574', 'GSM3892575', 'GSM3892576', 'num_of_cells']

    genes_info = pd.DataFrame(columns=columns)
    genes_info['Index'] = valid_genes_list
    
    num_of_cells = np.zeros(len(valid_genes_list))

    for i in range(len(files)):
        df = pd.read_csv(files[i])
        df_genes = list(df['Index'])
        sum_values=[]
        for gene in valid_genes_list:
            ind = df_genes.index(gene)
            df_gene = list(df.loc[ind, :])
            sum_=len(df_gene) - df_gene.count(0) -1
            sum_values.append(sum_)
            num_of_cells[valid_genes_list.index(gene)] += len(df_gene[1:]) 
        genes_info[columns[i+1]]=sum_values
    genes_info['num_of_cells'] = list(num_of_cells)
    genes_info.to_csv("Num_of_positive_per_gene.csv", index=False, header=columns)
    

### Spajanje datoteka, uklanjanje gena koji imaju broj pozitivnih celija < od nekog broja

In [14]:
def remove_genes_in_all_files(p):
    genes_info = pd.read_csv("Num_of_positive_per_gene.csv")
    columns = genes_info.columns[1:-1]
    
    num_of_positive_per_file = []
    for column in columns:
        l = list(genes_info[column])
        num_of_positive_per_file.append(l)

    genes = list(genes_info["Index"])
    num_of_cells = list(genes_info['num_of_cells'])
    
    index = []
    for i in range(len(genes)):
        sum_ = 0
        for j in range(len(num_of_positive_per_file)):
            sum_ += num_of_positive_per_file[j][i]
        if sum_ < num_of_cells[i]*0.01:
            index.append(i)
            
    print(len(index))
    genes_info=genes_info.drop(index)
    genes_info=genes_info.reset_index(drop=True)
    
    return genes_info

### Obrada podataka

In [8]:
files = ['GSM3892570_PBMC_DRESS1_filtered_gene_bc_matrices_h5.csv' , 'GSM3892571_PBMC_HV_filtered_gene_bc_matrices_h5.csv', 'GSM3892572_PBMC_T4_CTRL_filtered_gene_bc_matrices_h5.csv', 
       'GSM3892573_PBMC_T4_BACT_filtered_gene_bc_matrices_h5.csv', 'GSM3892574_DRESS_Day4_BACT_filtered_gene_bc_matrices_h5.csv', 
       'GSM3892575_DRESS_Day4_TOFA_filtered_gene_bc_matrices_h5.csv', 'GSM3892576_PBMC_POST2W_filtered_gene_bc_matrices_h5.csv']

valid_genes_list =list(pd.read_csv("Valid_genes.csv")['Index'])
valid_genes_list.sort()
#print(valid_genes_list)

num_of_positive = 500 
sum_of_positive = 1000
p =0.01

In [13]:
new_files= []

# ciscenje zasebnih datoteka i cuvanje preciscenih podataka u novim datotekama
#for i in range(len(files)):
i=0
file_name = files[i]
df = pd.read_csv(file_name)
    

    

In [14]:
cnum = len(df.columns)
df = rename_cells(df, cnum, file_name, i)
    
gnum = df.shape[0]
df = remove_invalid_genes(df, valid_genes_list, gnum)
    


In [15]:
df = remove_cells(df, cnum, num_of_positive, sum_of_positive)
    


7660
89


In [16]:
new_file_name = file_name[0: file_name.index('_')] + ".csv"
#new_files.append(new_file_name)
df.to_csv(new_file_name, index=False, header=df.columns)

In [19]:
df = pd.read_csv("GSM3892576.csv")
df

KeyboardInterrupt: 

In [9]:
new_files = ['GSM3892570.csv', 'GSM3892571.csv', 'GSM3892572.csv', 'GSM3892573.csv', 'GSM3892574.csv', 'GSM3892575.csv', 'GSM3892576.csv']

In [10]:
# pravi se datoteka sa informacijama o broju pozitivnih vrednosti celija za svaku datoteku po genu, 
# i ukupan broj celija za svaku datoteku
genes_info(valid_genes_list, new_files)


KeyboardInterrupt: 

In [16]:
df_genes_info = remove_genes_in_all_files(p)
# uzima se nova lista validnih gena iz preciscene datoteke koja sadrzi gene koji su validni na osnivu svih datoteka
valid_genes_list = list(df_genes_info['Index'])
len(valid_genes_list)
valid_genes_list

19254


['ENSG00000000419',
 'ENSG00000000457',
 'ENSG00000000460',
 'ENSG00000000938',
 'ENSG00000000971',
 'ENSG00000001036',
 'ENSG00000001084',
 'ENSG00000001167',
 'ENSG00000001460',
 'ENSG00000001461',
 'ENSG00000001497',
 'ENSG00000001561',
 'ENSG00000001629',
 'ENSG00000001630',
 'ENSG00000001631',
 'ENSG00000002016',
 'ENSG00000002330',
 'ENSG00000002549',
 'ENSG00000002586',
 'ENSG00000002587',
 'ENSG00000002822',
 'ENSG00000002834',
 'ENSG00000002919',
 'ENSG00000002933',
 'ENSG00000003056',
 'ENSG00000003147',
 'ENSG00000003393',
 'ENSG00000003400',
 'ENSG00000003402',
 'ENSG00000003509',
 'ENSG00000003756',
 'ENSG00000004059',
 'ENSG00000004139',
 'ENSG00000004142',
 'ENSG00000004399',
 'ENSG00000004455',
 'ENSG00000004468',
 'ENSG00000004478',
 'ENSG00000004487',
 'ENSG00000004534',
 'ENSG00000004660',
 'ENSG00000004700',
 'ENSG00000004766',
 'ENSG00000004777',
 'ENSG00000004779',
 'ENSG00000004799',
 'ENSG00000004864',
 'ENSG00000004866',
 'ENSG00000004897',
 'ENSG00000004961',


In [19]:
# prolazi se kroz nove datoteke i one se opet ciste na osnovu nove liste validnih gena
for i in range(len(new_files)):
    file_name = new_files[i]
    df = pd.read_csv(file_name)
    
    gnum = df.shape[0]
    df = remove_invalid_genes(df, valid_genes_list, gnum)
    
    cnum = len(df.columns)
    df = remove_cells(df, cnum, num_of_positive, sum_of_positive)
    
    df_sorted = df.sort_values(by=['Index'])
    df_sorted_ri=df_sorted.reset_index(drop=True)
    file_name = file_name[:file_name.index('.')] + '_.csv'
    df_sorted_ri.to_csv(file_name, index=False, header=df.columns)

7571
1
8648
0
4954
0
1999
0
1185
0
2888
0
6560
3
